In [22]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import yfinance as yf
from sklearn import covariance, cluster
from json import loads

In [23]:
# Input file containing company symbols 
input_file = '../aiwp-data/company_symbol_mapping.json'

# Load the company symbol map
with open(input_file, 'r') as f:
    company_symbols_map = loads(f.read())

symbols, names = np.array(list(company_symbols_map.items())).T
names
# for symbol in symbols[:3]:
#     print(symbol)

array(['Total', 'Exxon', 'Chevron', 'ConocoPhillips', 'Valero Energy',
       'Microsoft', 'IBM', 'Time Warner', 'Comcast', 'Cablevision',
       'Yahoo', 'Dell', 'HP', 'Amazon', 'Toyota', 'Canon', 'Mitsubishi',
       'Sony', 'Ford', 'Honda', 'Navistar', 'Northrop Grumman', 'Boeing',
       'Coca Cola', '3M', 'Mc Donalds', 'Pepsi', 'Kraft Foods', 'Kellogg',
       'Unilever', 'Marriott', 'Procter Gamble', 'Colgate-Palmolive',
       'General Electrics', 'Wells Fargo', 'JPMorgan Chase', 'AIG',
       'American express', 'Bank of America', 'Goldman Sachs', 'Apple',
       'SAP', 'Cisco', 'Texas instruments', 'Xerox', 'Lookheed Martin',
       'Wal-Mart', 'Walgreen', 'Home Depot', 'GlaxoSmithKline', 'Pfizer',
       'Sanofi-Aventis', 'Novartis', 'Kimberly-Clark', 'Ryder',
       'General Dynamics', 'Raytheon', 'CVS', 'Caterpillar',
       'DuPont de Nemours'], dtype='<U17')

In [24]:
# # Load the historical stock quotes 
# start_date = datetime.datetime(2003, 1, 1).strftime("%Y-%m-%d")
# end_date = datetime.datetime(2023, 9, 26).strftime("%Y-%m-%d")
# # downloading task
# for symbol in symbols:
#     df = yf.download('SPY',start=start_date, end = end_date)
#     df.to_csv(f'./stockdata/{symbol}.csv', encoding='utf-8', index=True)

In [25]:
df = pd.read_csv(f'./stockdata/MSFT.csv', index_col=0, parse_dates=True)
df.head(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2003-01-02,88.849998,91.300003,88.540001,91.070000,61.366577,44516300
2003-01-03,90.910004,91.379997,90.500000,91.349998,61.555252,32222600


In [26]:
# symbol_sample = symbols[:3]
# name_sample = names[:3]

In [27]:
# quotes = [pd.read_csv(f'./stockdata/{symbol}.csv', index_col=0, parse_dates=True) for symbol in symbol_sample]
quotes = [pd.read_csv(f'./stockdata/{symbol}.csv', index_col=0, parse_dates=True) for symbol in symbols]
# quotes

In [28]:
np.float = float

In [29]:
# # Extract opening and closing quotes
# opening_quotes = np.array([quote['Open'] for quote in quotes]).astype(np.float)
# closing_quotes = np.array([quote['Close'] for quote in quotes]).astype(np.float)
# # opening_quotes

In [30]:
# # Compute differences between opening and closing quotes 
# quotes_diff = closing_quotes - opening_quotes
# # quotes_diff

In [31]:
# X = quotes_diff.copy()
# # X.shape     # (3, 5218)
# # X

In [32]:
# # Normalize the data 
# X = quotes_diff.copy().T
# # X.shape     # (5218, 3)
# # type(X)


In [33]:
# X /= X.std(axis=0)
# # type(X)   # numpy.ndarray

In [34]:
# # Create a graph model 
# edge_model = covariance.GraphicalLassoCV()
# # edge_model

In [35]:
# # Train the model
# with np.errstate(invalid='ignore'):
#     edge_model.fit(X)

# # Build clustering model using Affinity Propagation model
# _, labels = cluster.affinity_propagation(edge_model.covariance_)
# num_labels = labels.max()
# num_labels

In [36]:
# # Print the results of clustering
# print('Clustering of stocks based on difference in opening and closing quotes:')
# for i in range(num_labels + 1):
#     print("Cluster", i+1, "==>", ', '.join(names[labels == i]))
#     # print("Cluster", i+1, "==>", ', '.join(name_sample[labels == i]))

In [37]:
# for quote in quotes:
#     print(quote.loc[(quote.index > start_date) & (quote.index <= end_date)].head(2))

In [38]:
def cluster_stocks(quotes, labels, names, start_date, end_date):
    quote_samples = [quote.loc[(quote.index > start_date) & (quote.index <= end_date)] for quote in quotes]
    p1_label, p2_label = labels
    p1_quotes = np.array([quote[p1_label] for quote in quote_samples]).astype(np.float)
    p2_quotes = np.array([quote[p2_label] for quote in quote_samples]).astype(np.float)
    quotes_diff = abs(p1_quotes - p2_quotes)
    X = quotes_diff.copy().T
    X /= X.std(axis=0)
    edge_model = covariance.GraphicalLassoCV()
    with np.errstate(invalid='ignore'):
        edge_model.fit(X)
    _, labels = cluster.affinity_propagation(edge_model.covariance_)
    num_labels = labels.max()
    print(f'Clustering of stocks based on difference in {p1_label} and {p2_label} quotes:')
    for i in range(num_labels + 1):
        print("Cluster", i+1, "==>", ', '.join(names[labels == i]))

In [39]:
start_date = '2007-01-01'
end_date   = '2023-09-01'
cluster_stocks(quotes, ['Open', 'High'], names, start_date, end_date)

Clustering of stocks based on difference in Open and High quotes:
Cluster 1 ==> Total, Exxon, Chevron, ConocoPhillips, Valero Energy, Microsoft, IBM, Time Warner, Comcast, Cablevision, Yahoo, Dell, HP, Amazon, Toyota, Canon, Mitsubishi, Sony, Ford, Honda, Navistar, Northrop Grumman
Cluster 2 ==> American express, Walgreen, Home Depot, GlaxoSmithKline, Kimberly-Clark, Ryder, Caterpillar, DuPont de Nemours
Cluster 3 ==> Boeing, Coca Cola, 3M, Mc Donalds, Pepsi, Kraft Foods, Kellogg, Unilever, Marriott, Procter Gamble, Colgate-Palmolive, General Electrics, Wells Fargo, JPMorgan Chase, AIG, Bank of America, Goldman Sachs, Apple, SAP, Cisco, Texas instruments, Xerox, Lookheed Martin, Wal-Mart, Pfizer, Sanofi-Aventis, Novartis, General Dynamics, Raytheon, CVS


In [40]:
start_date = '2010-01-01'
end_date   = '2023-09-01'
cluster_stocks(quotes, ['Open', 'Close'], names, start_date, end_date)

Clustering of stocks based on difference in Open and Close quotes:
Cluster 1 ==> Total, Goldman Sachs, Apple, SAP
Cluster 2 ==> Exxon, Chevron, ConocoPhillips, Valero Energy, Microsoft, Wells Fargo, JPMorgan Chase, AIG, American express, Bank of America
Cluster 3 ==> Sony, Unilever, Marriott, Procter Gamble, Colgate-Palmolive, General Electrics, Cisco, Texas instruments, Xerox, Lookheed Martin, Wal-Mart, Walgreen, Home Depot, GlaxoSmithKline, Pfizer, Sanofi-Aventis, Novartis, Kimberly-Clark, Ryder, General Dynamics, Raytheon, CVS, Caterpillar, DuPont de Nemours
Cluster 4 ==> IBM, Time Warner, Comcast, Cablevision, Yahoo, Dell, HP, Amazon, Toyota, Canon, Mitsubishi, Ford, Honda, Navistar, Northrop Grumman, Boeing, Coca Cola, 3M, Mc Donalds, Pepsi, Kraft Foods, Kellogg


/home/hung/.local/share/virtualenvs/data-science-k2Ak_Bao/lib/python3.11/site-packages/sklearn/covariance/_graph_lasso.py:183: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.216e-04
  warnings.warn(


In [43]:
start_date_1 = '2018-01-01'
end_date_1   = '2023-09-01'
cluster_stocks(quotes, ['Low', 'Close'], names, start_date_1, end_date_1)

Clustering of stocks based on difference in Low and Close quotes:
Cluster 1 ==> Total, Goldman Sachs, Apple, SAP
Cluster 2 ==> Exxon, Chevron, ConocoPhillips, Valero Energy, Microsoft, Wells Fargo, JPMorgan Chase, AIG, American express, Bank of America
Cluster 3 ==> Sony, Unilever, Marriott, Procter Gamble, Colgate-Palmolive, General Electrics, Cisco, Texas instruments, Xerox, Lookheed Martin, Wal-Mart, Walgreen, Home Depot, GlaxoSmithKline, Pfizer, Sanofi-Aventis, Novartis, Kimberly-Clark, Ryder, General Dynamics, Raytheon, CVS, Caterpillar, DuPont de Nemours
Cluster 4 ==> IBM, Time Warner, Comcast, Cablevision, Yahoo, Dell, HP, Amazon, Toyota, Canon, Mitsubishi, Ford, Honda, Navistar, Northrop Grumman, Boeing, Coca Cola, 3M, Mc Donalds, Pepsi, Kraft Foods, Kellogg


/home/hung/.local/share/virtualenvs/data-science-k2Ak_Bao/lib/python3.11/site-packages/sklearn/covariance/_graph_lasso.py:183: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.216e-04
  warnings.warn(


In [42]:
start_date_1 = '2006-01-01'
end_date_1   = '2023-09-01'
cluster_stocks(quotes, ['Low', 'High'], names, start_date_1, end_date_1)

Clustering of stocks based on difference in Low and High quotes:
Cluster 1 ==> Total, ConocoPhillips, Valero Energy, Goldman Sachs, Novartis, Kimberly-Clark, Ryder, General Dynamics, Raytheon, CVS, Caterpillar, DuPont de Nemours
Cluster 2 ==> Microsoft, IBM, Time Warner, Comcast, Cablevision, Yahoo, Dell, HP, Amazon, Toyota, Canon, Mitsubishi
Cluster 3 ==> Exxon, Chevron, Sony, Ford, Honda, Navistar, Northrop Grumman, Boeing, Coca Cola
Cluster 4 ==> 3M, Mc Donalds, Pepsi, Kraft Foods, Kellogg, Unilever, Marriott, Procter Gamble, Colgate-Palmolive, Bank of America, Apple, SAP, Cisco, Texas instruments, Xerox, Lookheed Martin, Wal-Mart, Walgreen
Cluster 5 ==> General Electrics, Home Depot, GlaxoSmithKline, Pfizer
Cluster 6 ==> Wells Fargo, JPMorgan Chase, AIG, American express, Sanofi-Aventis
